In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
import optuna
import torch
import random
import numpy as np

from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset

torch.set_float32_matmul_precision("high")
torch.use_deterministic_algorithms(True, warn_only=True)

torch.manual_seed(47)
random.seed(47)
np.random.seed(47)

2025-06-12 07:19:11.363 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /workspace/snn-encoder-test


In [3]:
# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

In [5]:
tuning_encoder = "tbr"

In [ ]:
import os

fine_study = optuna.load_study(
    study_name=f"model-fine-tuning-{tuning_encoder}-new",
    storage=os.environ["OPTUNA_CONN_STRING"],
)

In [ ]:
best_params = fine_study.best_params
best_trial = fine_study.best_trial

FrozenTrial(number=0, state=1, values=[0.19389110803604126], datetime_start=datetime.datetime(2025, 6, 11, 20, 18, 17, 515895), datetime_complete=datetime.datetime(2025, 6, 11, 20, 23, 28, 12372), params={'threshold': 0.05837571787591716, 'slope': 6.7082922718644795, 'beta': 0.6703615532512789, 'dropout_rate1': 0.3867650737818362, 'dropout_rate2': 0.2650897828875425, 'lr': 6.82947151662786e-05, 'weight_decay': 3.48210294726769e-06, 'scheduler_factor': 0.3434576235217472, 'scheduler_patience': 10, 'tbr_threshold': 0.7028430339186242}, user_attrs={'loss': 9.40915298461914, 'accuracy': 0.7941567301750183, 'f1': 0.804045557975769}, system_attrs={'fixed_params': {'threshold': 0.05837571787591716, 'slope': 6.7082922718644795, 'beta': 0.6703615532512789, 'dropout_rate1': 0.3867650737818362, 'dropout_rate2': 0.2650897828875425, 'lr': 6.82947151662786e-05, 'weight_decay': 3.48210294726769e-06, 'scheduler_factor': 0.3434576235217472, 'scheduler_patience': 10, 'tbr_threshold': 0.7028430339186242}

In [ ]:
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import OptimizerConfig
from eeg_snn_encoder.tuning import ENCODER_TUNING_FUNCTIONS

model_params: ModelConfig = {
    "threshold": best_params["threshold"],
    "slope": best_params["slope"],
    "beta": best_params["beta"],
    "dropout_rate1": best_params["dropout_rate1"],
    "dropout_rate2": best_params["dropout_rate2"],
}

optimizer_params: OptimizerConfig = {
    "lr": best_params["lr"],
    "weight_decay": best_params["weight_decay"],
    "scheduler_factor": best_params["scheduler_factor"],
    "scheduler_patience": best_params["scheduler_patience"],
}

encoder = ENCODER_TUNING_FUNCTIONS[tuning_encoder](best_trial) # type: ignore

In [ ]:
import gc

import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from loguru import logger
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler

from eeg_snn_encoder.config import MODELS_DIR
from eeg_snn_encoder.models.lightning import LitSeizureClassifier

logger.info(f"Starting {tuning_encoder}-encoder final evaluation")
logger.info(f"Params model:{model_params}, optimizer:{optimizer_params}, encoder:{encoder}")

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_results = []

generator = torch.manual_seed(42)

for fold, (train_val_ids, test_ids) in enumerate(kfold.split(dataset)): # type: ignore
    logger.info(f"Starting fold {fold + 1} of {kfold.get_n_splits()} {tuning_encoder}-encoder")

    train_ids, val_ids = train_test_split(
        train_val_ids, test_size=0.2, random_state=42, shuffle=True
    )

    train_sampler = SubsetRandomSampler(train_ids)
    val_sampler = SubsetRandomSampler(val_ids)
    test_sampler = SubsetRandomSampler(test_ids) # type: ignore

    trainloader = DataLoader(dataset, batch_size=512, sampler=train_sampler, generator=generator)
    valloader = DataLoader(dataset, batch_size=512, sampler=val_sampler, generator=generator)
    testloader = DataLoader(dataset, batch_size=512, sampler=test_sampler, generator=generator)

    lit_model = LitSeizureClassifier(
            model_config=model_params,
            optimizer_config=optimizer_params,
            spike_encoder=encoder,
        )

    trainer = pl.Trainer(
            max_epochs=100,
            accelerator="auto",
            devices="auto",
            strategy="auto",
            default_root_dir=MODELS_DIR / f"{tuning_encoder}_encoder",
            callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=5)],
            logger=False,
        )

    trainer.fit(lit_model, trainloader, valloader)
    trainer.test(lit_model, testloader, ckpt_path="best")

    test_loss = trainer.callback_metrics["test_loss"].item()
    test_acc = trainer.callback_metrics["test_acc"].item()
    test_precision = trainer.callback_metrics["test_precision"].item()
    test_recall = trainer.callback_metrics["test_recall"].item()
    test_f1 = trainer.callback_metrics["test_f1"].item()
    test_mse = trainer.callback_metrics["test_mse"].item()
    test_total_spikes = trainer.callback_metrics["test_total_spikes"].item()

    logger.info(
        f"{tuning_encoder}-encoder ",
        f"Fold {fold + 1} - Test Loss: {test_loss:.4f}, "
        f"Test Accuracy: {test_acc:.4f}, "
        f"Test Precision: {test_precision:.4f}, "
        f"Test Recall: {test_recall:.4f}, "
        f"Test F1: {test_f1:.4f}, "
        f"Test MSE: {test_mse:.4f}, "
        f"Test Total Spikes: {test_total_spikes:.4f}",
    )

    fold_results.append(
        {
            "fold": fold,
            "test_loss": test_loss,
            "test_acc": test_acc,
            "test_precision": test_precision,
            "test_recall": test_recall,
            "test_f1": test_f1,
            "test_mse": test_mse,
            "test_total_spikes": test_total_spikes,
        }
    )

    del lit_model
    del trainer
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import json

import pandas as pd

from eeg_snn_encoder.config import REPORTS_DIR

results_df = pd.DataFrame(fold_results)
results_df.set_index("fold", inplace=True)

results_df.to_csv(REPORTS_DIR / f"{tuning_encoder}_model_results_new.csv", index=True)

params_file = REPORTS_DIR / f"{tuning_encoder}_model_params_new.json"

params_file.write_text(
    json.dumps(
        {
            "model_params": model_params,
            "optimizer_params": optimizer_params,
            "encoder_params": str(encoder),
        },
        indent=4,
    )
)
logger.info(f"Results and parameters saved to {REPORTS_DIR} for {tuning_encoder}-encoder")